# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')

In [5]:
# Create cursor object 
cur = conn.cursor()

In [6]:
# Execute a query to retrieve the tables names 
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# fetch all table names
tables = cur.fetchall()

In [7]:
# display the table names 
for table in tables:
    print(table[0])

productlines
offices
customers
employees
orderdetails
orders
payments
products


## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [9]:
# Your code here
cur.execute(
    """
    SELECT 
        customerNumber,
        contactLastName,
        contactFirstName
    FROM
        customers
    WHERE 
        customerNumber IN (
        SELECT 
            customerNumber
        FROM
            orders
        WHERE 
            orderDate = '2003-01-31'
        );
""")

# fetch the data 
rows = cur.fetchall()

# display the data 
for row in rows:
    print(row)

(141, 'Freyre', 'Diego ')


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [12]:
# Your code here
# define the query 
query = """
    SELECT products.productName, COUNT(orders.orderNumber) AS total_orders
    FROM products 
    JOIN orderdetails  ON products.productCode = orderDetails.productCode
    JOIN orders  ON orderDetails.orderNumber = orders.orderNumber
    GROUP BY products.productName
    ORDER BY total_orders DESC;
"""

# execute the query 
cur.execute(query)

# fetch the results 
results = cur.fetchall()

# display the results 
for row in results:
    print(f"{row[0]}: {row[1]}")

1992 Ferrari 360 Spider red: 53
P-51-D Mustang: 28
HMS Bounty: 28
F/A 18 Hornet 1/72: 28
Diamond T620 Semi-Skirted Tanker: 28
Corsair F4U ( Bird Cage): 28
Boeing X-32A JSF: 28
American Airlines: MD-11S: 28
American Airlines: B767-300: 28
America West Airlines B757-200: 28
ATA: B757-300: 28
2003 Harley-Davidson Eagle Drag Bike: 28
2002 Suzuki XREO: 28
1999 Yamaha Speed Boat: 28
1998 Chrysler Plymouth Prowler: 28
1997 BMW R 1100 S: 28
1997 BMW F650 ST: 28
1996 Peterbilt 379 Stake Bed with Outrigger: 28
1996 Moto Guzzi 1100i: 28
1982 Camaro Z28: 28
1980’s GM Manhattan Express: 28
1980s Black Hawk Helicopter: 28
1972 Alfa Romeo GTA: 28
1970 Dodge Coronet: 28
1969 Harley Davidson Ultimate Chopper: 28
1964 Mercedes Tour Bus: 28
1962 Volkswagen Microbus: 28
1962 LanciaA Delta 16V: 28
1960 BSA Gold Star DBD34: 28
1958 Setra Bus: 28
1958 Chevy Corvette Limited Edition: 28
1957 Chevy Pickup: 28
1954 Greyhound Scenicruiser: 28
1952 Alpine Renault 1300: 28
1941 Chevrolet Special Deluxe Cabriolet: 

In [14]:
# Your code here
query = """
    SELECT
        products.productName, (
        SELECT
            COUNT(*)
            FROM orderdetails 
            WHERE orderDetails.productCode = products.productCode
        ) AS total_orders
        FROM products 
        ORDER BY total_orders DESC;
"""

# execute the Query 
cur.execute(query)

# fetch the results 
myresult = cur.fetchall()

# display the results 
# print(myresult)
for row in myresult:
    print(f"{row[0]}: {row[1]}")

1992 Ferrari 360 Spider red: 53
1969 Harley Davidson Ultimate Chopper: 28
1952 Alpine Renault 1300: 28
1996 Moto Guzzi 1100i: 28
2003 Harley-Davidson Eagle Drag Bike: 28
1972 Alfa Romeo GTA: 28
1962 LanciaA Delta 16V: 28
1958 Setra Bus: 28
2002 Suzuki XREO: 28
1957 Chevy Pickup: 28
1940 Ford Pickup Truck: 28
1937 Lincoln Berline: 28
1936 Mercedes-Benz 500K Special Roadster: 28
1980s Black Hawk Helicopter: 28
1998 Chrysler Plymouth Prowler: 28
1964 Mercedes Tour Bus: 28
1932 Model A Ford J-Coupe: 28
1926 Ford Fire Engine: 28
P-51-D Mustang: 28
1936 Harley Davidson El Knucklehead: 28
1928 Mercedes-Benz SSK: 28
1913 Ford Model T Speedster: 28
1934 Ford V8 Coupe: 28
1999 Yamaha Speed Boat: 28
18th Century Vintage Horse Carriage: 28
1917 Maxwell Touring Car: 28
1941 Chevrolet Special Deluxe Cabriolet: 28
1940s Ford truck: 28
1939 Cadillac Limousine: 28
1970 Dodge Coronet: 28
1997 BMW R 1100 S: 28
1928 British Royal Navy Airplane: 28
1939 Chevrolet Deluxe Coupe: 28
1960 BSA Gold Star DBD34: 

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [17]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
query="""
SELECT 
    products.productName,
    (
        SELECT COUNT(DISTINCT orders.customerNumber)
        FROM orders
        WHERE orders.orderNumber IN (
            SELECT orderNumber
            FROM orderDetails
            WHERE orderDetails.productCode = products.productCode
        )
    ) AS totalPeopleOrdered
FROM 
    products
ORDER BY 
    totalPeopleOrdered DESC;
    """


In [18]:
pd.read_sql(query,conn)

,productName,totalPeopleOrdered
0,1992 Ferrari 360 Spider red,40
1,1952 Alpine Renault 1300,27
2,1972 Alfa Romeo GTA,27
3,1934 Ford V8 Coupe,27
4,Boeing X-32A JSF,27
...,...,...
105,1949 Jaguar XK 120,18
106,1952 Citroen-15CV,18
107,1969 Chevrolet Camaro Z28,18
108,2002 Chevy Corvette,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [19]:
# Your code here
query="""
SELECT 
    employees.employeeNumber,
    employees.firstName,
    employees.lastName,
    offices.city,
    employees.officeCode
FROM 
    employees
JOIN 
    offices ON employees.officeCode = offices.officeCode
WHERE 
    employees.employeeNumber IN (
        SELECT 
            employeeNumber
        FROM 
            employees
        WHERE 
            employeeNumber IN (
                SELECT 
                    customers.salesRepEmployeeNumber
                FROM 
                    customers
                JOIN 
                    orders ON customers.customerNumber = orders.customerNumber
                WHERE 
                    orders.orderNumber IN (
                        SELECT 
                            orderNumber
                        FROM 
                            orderDetails
                        WHERE 
                            productCode IN (
                                SELECT 
                                    productCode
                                FROM 
                                    orderDetails
                                JOIN 
                                    orders ON orderDetails.orderNumber = orders.orderNumber
                                GROUP BY 
                                    productCode
                                HAVING 
                                    COUNT(DISTINCT orders.customerNumber) < 20
                            )
                    )
                GROUP BY 
                    customers.salesRepEmployeeNumber
            )
        GROUP BY 
            employeeNumber
    )
ORDER BY 
    employees.lastName, employees.firstName;
    """


In [20]:
pd.read_sql(query,conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1337,Loui,Bondur,Paris,4
1,1501,Larry,Bott,London,7
2,1401,Pamela,Castillo,Paris,4
3,1188,Julie,Firrelli,Boston,2
4,1611,Andy,Fixter,Sydney,6
5,1702,Martin,Gerard,Paris,4
6,1370,Gerard,Hernandez,Paris,4
7,1165,Leslie,Jennings,San Francisco,1
8,1504,Barry,Jones,London,7
9,1612,Peter,Marsh,Sydney,6


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [29]:
# Your code here
query="""
SELECT
    employees.employeeNumber,
    employees.firstName,
    employees.lastName,
    (
        SELECT COUNT(*)
        FROM customers
        WHERE customers.salesRepEmployeeNumber = employees.employeeNumber
    ) AS numCustomers
FROM
    employees
WHERE
    employees.employeeNumber IN (
        SELECT
            customers.salesRepEmployeeNumber
        FROM
            customers
        GROUP BY
            customers.salesRepEmployeeNumber
        HAVING
            AVG(customers.creditLimit) > 15000
    )
ORDER BY
    employees.lastName, employees.firstName;
    """


In [28]:
pd.read_sql(query,conn)

,employeeNumber,firstName,lastName,numCustomers
0,1337,Loui,Bondur,6
1,1501,Larry,Bott,8
2,1401,Pamela,Castillo,10
3,1188,Julie,Firrelli,6
4,1611,Andy,Fixter,5
5,1702,Martin,Gerard,6
6,1370,Gerard,Hernandez,7
7,1165,Leslie,Jennings,6
8,1504,Barry,Jones,9
9,1612,Peter,Marsh,5


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!